In [1]:
from snappy import ProductIO
from snappy import jpy
from snappy import GPF

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import numpy as np

from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from datetime import timedelta
from datetime import datetime

import zipfile
import os 
import time
import shutil
import pickle

In [2]:
# télecharge les images correspndantes au critère de recherche

def download_olci_data(user,password,url,footprint,datestart,dateend,zip_file_path,image_number):
    
    #user = 'jonathanc'
    #password = 'Tutorat2019'
    #api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
    #url = 'https://coda.eumetsat.int'
    api = SentinelAPI(user, password, url)

    #footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
    #date = ('20191001','NOW')
    products = api.query(footprint,
                         date = (datestart,dateend),
                         platformname = 'Sentinel-3',
                         area_relation='Intersects', 
                         producttype='OL_2_WFR___',
                         order_by='beginposition',
                         #timeliness='Non Time Critical',
                         instrumentshortname = 'OLCI',
                         onlinequalitycheck = 'PASSED',
                         productlevel = 'L2')
    # convert to Pandas DataFrame
    products_df = api.to_dataframe(products)
    # sort and limit to first 5 sorted products
    products_df_sorted = products_df.sort_values(['beginposition'], ascending=[True])
    products_df_sorted = products_df_sorted.head(image_number)
    # download sorted and reduced products
    api.download_all(products_df_sorted.index,directory_path = zip_file_path+"/")
    print zip_file_path+"/"
    #zip_file_path = "/home/courtois/jupyter-sentinel"
    file_list = os.listdir(zip_file_path+"/")
    # select the directory where the files are and where you want to unzip it
    abs_path = []
    # list for the .zip file to unzip
    c=0
    for a in file_list:
        if a.endswith(".zip"):  # select only the .zip files in the folder
            x = zip_file_path+'/'+a
            c += 1
            print x
            abs_path.append(x)  # put the .zip file in the list
    for f in abs_path:
        zip=zipfile.ZipFile(f)
        zip.extractall(zip_file_path)  # unzip the file in the zip_file_path
        os.remove(f)   # delet the .zip file
        
    print("File in :" + zip_file_path)
    print("DONE")

In [3]:
# donne une liste de fichier en affichant que les nouveaux fichiers si images_list n'es pas une liste vide.
def list_product(images_path, images_list):
    file_list = os.listdir(images_path +"/")
    # definition du chemin d'accès

    # list contenant tout les noms de fichier 
    
    add_name_list = []
    for a in file_list:
        x = images_path+'/'+a
        #print x
        if a.endswith(".SEN3") and not(x in images_list):
            add_name_list.append(x)  # Dézip
    return add_name_list

# exemple d'utilisation
#images_path = "/media/DATA-4To/Sentinel"
#images_list = []
#images_list += list_product(images_path, images_list)
#print len(images_list),"files"

In [4]:
# Créé un filtre (liste d'index) de taille déterminé en partant d'un point de coordonée
def cubixGeoCut(point_lon,point_lat,sub_product,band_product,width, height):
    Lon = sub_product.getBand('longitude');
    Lat = sub_product.getBand('latitude');
    # on obtient les matrices de longitude et latitude
    
    band_Width = band_product.getRasterWidth()
    band_Height = band_product.getRasterHeight()
    
    Lon_d = np.zeros(band_Width*band_Height, dtype = np.float32)
    Lon.readPixels(0,0,band_Width,band_Height,Lon_d)
    Lon_d.shape = (band_Height, band_Width) 

    Lat_d = np.zeros(band_Width*band_Height, dtype = np.float32)
    Lat.readPixels(0,0,band_Width,band_Height,Lat_d)
    Lat_d.shape = (band_Height, band_Width) 

    # on chercher le point le plus  proche de la coordonnées souhaitée
    min_lon_mat = abs(Lon_d - point_lon)
    min_lat_mat = abs(Lat_d - point_lat)

    #obtention de l'indice du minimum
    min_coord = np.dstack((min_lon_mat,min_lat_mat))
    min_coord = np.dstack((min_coord,min_coord[:,:,0]+min_coord[:,:,1]))
    minc = np.unravel_index(np.argmin(min_coord[:,:,2], axis=None), min_coord[:,:,0].shape)

    # création de la matrice d'index de taille 125*125
    x=0
    submatidx = np.zeros((height*width,2),dtype=int)

    for i in range(minc[0],minc[0]+height):
        for j in range(minc[1],minc[1]+width):
            submatidx[x] = (i,j)
            x = x+1
    return submatidx

In [5]:
# créé une liste des noms de bandes
def reflectance_band_names():
    reflect_band_names = []
    #recupréation du nom des bandes interssantes
    for i in range(1,10):
        reflect_band_names.append(str('Oa0'+str(i)+'_reflectance'))
    reflect_band_names.append(str('Oa10_reflectance'))
    reflect_band_names.append(str('Oa11_reflectance'))
    reflect_band_names.append(str('Oa12_reflectance'))
    reflect_band_names.append(str('Oa16_reflectance'))
    reflect_band_names.append(str('Oa17_reflectance'))
    reflect_band_names.append(str('Oa18_reflectance'))
    reflect_band_names.append(str('Oa21_reflectance'))
    #print(reflect_band_names)
    return reflect_band_names

In [6]:
def date_finder(name,tag):
    # recupération de l'index du debut de la date dans le nom ex : ...WFR____20191026T100707...
    date_index = name.find(tag)+len(tag)
    # extraction de la date : 20191026
    date_str = name[date_index:date_index+8]
    # mise en forme jour, mois, années : ['26', '10', '2019']
    date = [date_str,date_str[6:8],date_str[4:6],date_str[0:4]]
    return date

In [7]:
def data_image_builder(images_list, footprint, point_coord, image_size,cloud_cover=5, print_match=False):
    # variables init
    imageData = []
    prog = 1
    point_lon = point_coord[0]
    point_lat = point_coord[1] 
    height_reduit = image_size[0]
    width_reduit = image_size[1]
    # réglage de la géométrie et des sous produit
    SubsetOp = jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')
    WKTReader =jpy.get_type('com.vividsolutions.jts.io.WKTReader')
    geometry = WKTReader().read(footprint) #préparation de la géométrie de la ZOI 
    
    # liste des noms de bande
    reflect_band_names = reflectance_band_names()
    
    for path_name in images_list:
        print(path_name)

        # chargement du product
        product = ProductIO.readProduct(path_name)

        HashMap = jpy.get_type('java.util.HashMap')  
        parameters = HashMap()

        # generation d'un sous produit autour de la zoi
        op = SubsetOp()
        op.setSourceProduct(product)
        op.setGeoRegion(geometry)
        sub_product = op.getTargetProduct()

        # Obtention d'une liste de bandes 
        reflect_band_data = []
        timesaver = True
        for name in reflect_band_names:

            start_time = time.time()

            reflect_band_data.append(sub_product.getBand(name)) 
            if (timesaver):
                # Récupération de la tailles des bandes
                Width = reflect_band_data[0].getRasterWidth()
                Height = reflect_band_data[0].getRasterHeight()
                print "Image full scale : Width:",Width," ; Height:",Height
                if not(Width>500 and Height>500):

                    print("--- %s seconds size break---" % (time.time() - start_time))
                    break
                else:
                    timesaver = False
        # outils de filtrage :
        # on filtre si : la taille ne correspond pas OU si la couerture T/C est > 10 %
        remove = True

        # test de taille minimum pour l'image
        if (Width>500 and Height>500):
            # extraire les données de reflectances de toutes les bandes dans une matrice
            # definition des paramètres codant la découpe de l'image dans la ZOI
            # point_lon = 4.654875873875496 
            # point_lat = 43.41241369162426
            # height_reduit = 125
            # width_reduit = 125
            # matrice de donnée
            # IL FAUDRA surement verifier les index pour ne pas depasser les capacitée de la matrice ######################
            #print "len(reflect_band_data)",len(reflect_band_data)
            
            data_reflect = np.empty([len(reflect_band_data), width_reduit, height_reduit ])
            # etablissement de la list d'indice qui permetera de reduire la matrice d'image de la ZOI
            submatidx = cubixGeoCut(point_lon,point_lat,sub_product,reflect_band_data[0],width_reduit, height_reduit )
            #print("submatidx",submatidx)

            multiSpectralMatrixTmp = []
            for i in range(0,len(reflect_band_data)):

                # extraction des données 
                reflect_data = np.zeros(Width*Height, dtype = np.float32)
                reflect_band_data[i].readPixels(0,0,Height,Width,reflect_data)
                reflect_data.shape = Width, Height

                # reduction de la donnée dans la zone definit
                data_reflect[i] = reflect_data[submatidx[:,0],submatidx[:,1]].reshape(height_reduit,width_reduit)

                # afficher la coverture terrestre et nuageuse
                c=0
                for j in range(0,data_reflect[i].shape[0]):
                    for k in range(0,data_reflect[i].shape[1]):
                        if (data_reflect[i][j,k] > 1):
                            c = c+1
                c = (c*100)/(data_reflect[i].shape[0]*data_reflect[i].shape[1])
                if (i == 0) : print "cloud/land coverage:",c,"%"

                if (c<=cloud_cover):
                    # ici on est dans la condition couverture L/C <= 5% ET taille de l'image de base convenable
                    remove = False
                    multiSpectralMatrixTmp.append(data_reflect[i])
                else:
                    i = len(reflect_band_data)+1
                    print("--- %s seconds reading band break ---" % (time.time() - start_time))
                    break

                # plot et subplot etc ...
                if (i==5 and not(remove) and print_match):

                    # plot
                    plt.figure(figsize=(8, 8))                 # adjusting the figure window size
                    fig = plt.imshow(data_reflect[i], cmap = cm.gray)  #matplotlib settings for the current image
                    fig.axes.get_xaxis().set_visible(False)
                    fig.axes.get_yaxis().set_visible(False)
                    plt.show()
                    # plot
                    plt.figure(figsize=(8, 8))                 # adjusting the figure window size
                    fig = plt.imshow(reflect_data, cmap = cm.gray)  #matplotlib settings for the current image
                    fig.axes.get_xaxis().set_visible(False)
                    fig.axes.get_yaxis().set_visible(False)
                    plt.show()

        product.closeIO() 
        # si l'iùage n'as pas passée les conditions ou si elle ne possèdent pas ses 16 bandes exploitable on la supprime
        if remove or (len(multiSpectralMatrixTmp)!=16):
            print('removed')
            shutil.rmtree(path_name)
        else :
            # ajout de la date en fin de list
            date = date_finder(path_name,'WFR____')
            multiSpectralMatrixTmp.append(date)
            imageData.append(multiSpectralMatrixTmp)
            print("--- %s seconds Processing done ---" % (time.time() - start_time))
        print prog,"/",len(images_list)
        prog +=1
    return imageData

In [8]:
def add_one_day_imlistata(imageList):
    date_list = []
    for i in range(len(imageList)): 
        date_list.append(date_finder(imageList[i],'WFR____')[0])
    date_list.sort(reverse = True)
    
    addtime = (datetime.strptime(date_list[0], '%Y%m%d')+timedelta(hours=+1)).__str__()
    addtime = addtime[0:4]+addtime[5:7]+addtime[8:10]

    return addtime

In [9]:
user = 'jonathanc'
password = 'Tutorat2019'
url = 'https://coda.eumetsat.int'
footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
datestart = ('20190101')
dateend = ('NOW')
images_path = "/media/DATA-4To/Sentinel"
image_number = 1000
Download_step = 10

point_coord = (4.654875873875496,43.41241369162426)
image_size = (125,125)
        
image_segmentation = 0
images_list = []
imageData = []
stopDL = False

while (image_number-image_segmentation > 0):
    # segmentation des données pour le téléchargement
    image_segmentation += Download_step
    
    if not(stopDL):
        # Téléchargement des données
        download_olci_data(user,password,url,footprint,datestart,dateend,images_path,Download_step)

        # Récupération des noms de fichiers fraichement téléchargé dans un segment
        new_images_list = list_product(images_path , images_list)
    
    if new_images_list != [] :
        # Mise à jour de la liste total des fichiers téléchargés
        images_list += new_images_list;

        # Création d'une liste de matrice de dimension image_size_x,image_size_y,nombre_de_raie
        imageData += data_image_builder(new_images_list, footprint, point_coord, image_size)

        # Récupération de la date du dernier fichier téléchargé 
        #  pour definir la date des prochaines images à télécharger
        datestart = add_one_day_imlistata(new_images_list)

        # mise en format datetime des date de debut et fin pour pouvoir les comparer
        start_datetime = datetime.strptime(datestart, '%Y%m%d')

        if (dateend == ('NOW')):
            date_end = datetime.today()
        else:
            date_end = datetime.strptime(dateend, '%Y%m%d')

        print (start_datetime,date_end)
        if (start_datetime > date_end):
            image_segmentation = image_number+1
    else:
        # il n'y à plus de fichier à télécharger on stop le télechargement
        stopDL = True 
        image_segmentation = image_number
    
    print(" ------------ %d download / %d ------------" % (image_segmentation, image_number))
    
print("TASK DONE\nCreating ImageDataMatrix file ...")  
with open("ImageDataMatrix.txt", "wb") as fp:   #Pickling
    pickle.dump(imageData, fp)
print"DONE: ImageDataMatrix.txt saved in directory:\n"+images_path

#with open("ImageDataMatrix.txt", "rb") as fp:   # Unpickling
#    b = pickle.load(fp)

Querying products: 100%|██████████| 499/499 [00:04<00:00, 111.70 products/s]
Downloading: 100%|██████████| 236M/236M [00:18<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 239M/239M [00:19<00:00, 12.2MB/s] 
Downloading: 100%|██████████| 209M/209M [00:16<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 186M/186M [00:14<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 163M/163M [00:13<00:00, 12.3MB/s] 
Downloading: 100%|██████████| 155M/155M [00:12<00:00, 12.3MB/s] 
Downloading: 100%|██████████| 157M/157M [00:12<00:00, 12.3MB/s] 
Downloading: 100%|██████████| 202M/202M [00:16<00:00, 12.2MB/s] 
ERROR:sentinelsat.SentinelAPI:There was an error downloading e4baa0f2-68fa-42f1-bd08-c5a9f0ca7b24
Traceback (most recent call last):
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 515, in download_all
    product_info = self.download(product_id, directory_path, checksum)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py",

/media/DATA-4To/Sentinel/
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190101T093149_20190101T093449_20190102T171625_0179_039_364_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190107T101643_20190107T101943_20190108T172812_0179_040_065_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190113T092037_20190113T092337_20190114T175859_0179_040_150_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190108T095032_20190108T095332_20190109T193326_0179_040_079_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190103T102027_20190103T102327_20190104T192049_0179_040_008_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190105T092805_20190105T093105_20190106T175913_0179_040_036_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190109T092421_20190109T092721_20190110T180451_0179_040_093_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190111T101259_20190111T101559_20190112T182931_0179_040_122_2160

Querying products: 100%|██████████| 490/490 [00:06<00:00, 72.48 products/s]
Downloading: 100%|██████████| 152M/152M [00:12<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 214M/214M [00:17<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 234M/234M [00:18<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 210M/210M [00:16<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 197M/197M [00:15<00:00, 12.4MB/s] 
Downloading: 100%|██████████| 166M/166M [00:13<00:00, 12.2MB/s] 
ERROR:sentinelsat.SentinelAPI:There was an error downloading 4c55b2b1-cbc9-4408-adc3-63a83233b6c6
Traceback (most recent call last):
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 515, in download_all
    product_info = self.download(product_id, directory_path, checksum)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 458, in download
    product_info['url'], temp_path, self.session, product_info['size'])
  File "/home/courtois/anaconda2/lib

/media/DATA-4To/Sentinel/
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190123T100147_20190123T100447_20190128T184230_0179_040_293_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190124T093536_20190124T093836_20190125T182811_0179_040_307_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190122T102758_20190122T103058_20190123T235913_0179_040_279_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190115T100915_20190115T101215_20190116T193424_0179_040_179_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190113T092037_20190113T092337_20190114T175859_0179_040_150_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190126T102413_20190126T102713_20190127T182152_0179_040_336_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190119T100531_20190119T100831_20190120T190719_0179_040_236_2160_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/S3A_OL_2_WFR____20190116T094304_20190116T094604_20190117T192412_0179_040_193_2160

Querying products: 100%|██████████| 481/481 [00:08<00:00, 56.53 products/s]
Downloading: 100%|██████████| 169M/169M [00:13<00:00, 12.4MB/s] 
Downloading: 100%|██████████| 168M/168M [00:13<00:00, 12.1MB/s] 
Downloading: 100%|██████████| 177M/177M [00:14<00:00, 12.5MB/s] 
Downloading: 100%|██████████| 215M/215M [00:16<00:00, 12.9MB/s] 
Downloading: 100%|██████████| 177M/177M [00:15<00:00, 11.6MB/s] 
Downloading: 100%|██████████| 110M/110M [00:09<00:00, 11.9MB/s] 
Downloading: 100%|██████████| 275M/275M [00:21<00:00, 12.6MB/s] 
Downloading: 100%|██████████| 212M/212M [00:16<00:00, 12.8MB/s] 
Downloading: 100%|██████████| 211M/211M [00:16<00:00, 12.6MB/s] 
Downloading: 100%|██████████| 215M/215M [00:17<00:00, 12.0MB/s] 


...

ERROR:sentinelsat.SentinelAPI:There was an error downloading f6dc7cc6-0e27-433c-a9c0-bfe6a99c85d9
Traceback (most recent call last):
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 515, in download_all
    product_info = self.download(product_id, directory_path, checksum)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 417, in download
    product_info = self.get_product_odata(id)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 382, in get_product_odata
    response = self.session.get(url, auth=self.session.auth)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 546, in get
    return self.request('GET', url, **kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-pac

ERROR:sentinelsat.SentinelAPI:There was an error downloading bb3f7829-60c9-4d8a-8963-a5614ead03fd
Traceback (most recent call last):
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 515, in download_all
    product_info = self.download(product_id, directory_path, checksum)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 417, in download
    product_info = self.get_product_odata(id)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 382, in get_product_odata
    response = self.session.get(url, auth=self.session.auth)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 546, in get
    return self.request('GET', url, **kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-pac

ERROR:sentinelsat.SentinelAPI:There was an error downloading 413224b2-6f4f-4f38-b92c-0cbe4ebc795f
Traceback (most recent call last):
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 515, in download_all
    product_info = self.download(product_id, directory_path, checksum)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 417, in download
    product_info = self.get_product_odata(id)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 382, in get_product_odata
    response = self.session.get(url, auth=self.session.auth)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 546, in get
    return self.request('GET', url, **kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-pac

ERROR:sentinelsat.SentinelAPI:There was an error downloading 413224b2-6f4f-4f38-b92c-0cbe4ebc795f
Traceback (most recent call last):
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 515, in download_all
    product_info = self.download(product_id, directory_path, checksum)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 417, in download
    product_info = self.get_product_odata(id)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/sentinelsat/sentinel.py", line 382, in get_product_odata
    response = self.session.get(url, auth=self.session.auth)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 546, in get
    return self.request('GET', url, **kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/courtois/anaconda2/lib/python2.7/site-pac

ConnectionError: ('Connection aborted.', error("(104, 'ECONNRESET')",))